In [1]:
import pandas as pd

In [2]:
odds = pd.read_csv('champions league odds - odds.csv')

In [3]:
odds.head()

,hometeam,awayteam,season,homewin,draw,awaywin
0,AS Roma,Liverpool,2018,0.367839,0.240685,0.391476
1,Real Madrid,Bayern Munich,2018,0.407064,0.237869,0.355067
2,Bayern Munich,Real Madrid,2018,0.487331,0.238888,0.273781
3,Liverpool,AS Roma,2018,0.642321,0.211442,0.146237
4,Bayern Munich,Sevilla,2018,0.764948,0.142686,0.092366


In [15]:
odds['hometeam'] = odds['hometeam'].str.strip()
odds['awayteam'] = odds['awayteam'].str.strip()

In [16]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 6 columns):
hometeam    324 non-null object
awayteam    324 non-null object
season      324 non-null int64
homewin     322 non-null float64
draw        322 non-null float64
awaywin     322 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 15.3+ KB


In [17]:
len(odds)

324

In [18]:
results = pd.read_csv('results.csv')

In [19]:
results.head()

,season,round,tie,winner,pk,agr,aet,result
0,2008,first,arsenal-milan,arsenal,False,False,False,arsenal (2-0) milan
1,2008,first,celtic-barcelona,barcelona,False,False,False,celtic (2-4) barcelona
2,2008,first,fenerbahce-sevilla,fenerbahce,True,False,True,"fenerbahce (5-5 aet) sevilla, fenerbahce won o..."
3,2008,first,liverpool-inter,liverpool,False,False,False,liverpool (3-0) inter
4,2008,first,lyon-united,united,False,False,False,lyon (1-2) united


In [20]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
season    162 non-null int64
round     162 non-null object
tie       162 non-null object
winner    162 non-null object
pk        162 non-null bool
agr       162 non-null bool
aet       162 non-null bool
result    162 non-null object
dtypes: bool(3), int64(1), object(4)
memory usage: 6.9+ KB


In [21]:
len(results)

162

In [22]:
with open('teamcrosswalk.psv') as f:
    lines = f.readlines()
    xwalk = {l.split('|')[1]: l.split('|')[0] for l in lines[1:]}

In [23]:
xwalk

{'milan': 'AC Milan',
 'apoel': 'APOEL',
 'roma': 'AS Roma',
 'ajax': 'Ajax',
 'arsenal': 'Arsenal',
 'atletico': 'Atl. Madrid',
 'barcelona': 'Barcelona',
 'basel': 'Basel',
 'leverkusen': 'Bayer Leverkusen',
 'bayern': 'Bayern Munich',
 'benfica': 'Benfica',
 'besiktas': 'Besiktas',
 'bordeaux': 'Bordeaux',
 'cska': 'CSKA Moscow',
 'celtic': 'Celtic',
 'chelsea': 'Chelsea',
 'dortmund': 'Dortmund',
 'kyiv': 'Dyn. Kyiv',
 'copenhagen': 'FC Copenhagen',
 'porto': 'FC Porto',
 'fenerbahce': 'Fenerbahce',
 'fiorentina': 'Fiorentina',
 'galatasaray': 'Galatasaray',
 'gent': 'Gent',
 'inter': 'Inter',
 'juventus': 'Juventus',
 'leicester': 'Leicester',
 'liverpool': 'Liverpool',
 'lyon': 'Lyon',
 'malaga': 'Malaga',
 'city': 'Manchester City',
 'united': 'Manchester Utd',
 'marseille': 'Marseille',
 'monaco': 'Monaco',
 'napoli': 'Napoli',
 'olympiacos': 'Olympiakos Piraeus',
 'psv': 'PSV',
 'panathinaikos': 'Panathinaikos',
 'psg': 'Paris SG',
 'madrid': 'Real Madrid',
 'schalke': 'Schalk

In [24]:
final = []
for d in results.to_dict(orient='records'):
    teams = d['tie'].split('-')
    final.append(
        dict(
            season=d['season'],
            tie=d['tie'],
            hometeam=xwalk[teams[0]],
            awayteam=xwalk[teams[1]],
            game=1
        )
    )
    final.append(
        dict(
            season=d['season'],
            tie=d['tie'],
            hometeam=xwalk[teams[1]],
            awayteam=xwalk[teams[0]],
            game=2
        )
    )

In [30]:
gameoddsxwalk = pd.DataFrame(final).merge(odds)[['hometeam','awayteam','game','season','tie','homewin','draw','awaywin']]

In [34]:
gameoddsxwalk.head()

,hometeam,awayteam,game,season,tie,homewin,draw,awaywin
0,Arsenal,AC Milan,1,2008,arsenal-milan,0.452474,0.314634,0.232891
1,AC Milan,Arsenal,2,2008,arsenal-milan,0.428566,0.317081,0.254352
2,Celtic,Barcelona,1,2008,celtic-barcelona,0.234004,0.302405,0.463592
3,Barcelona,Celtic,2,2008,celtic-barcelona,0.821256,0.146369,0.032374
4,Fenerbahce,Sevilla,1,2008,fenerbahce-sevilla,0.342100,0.309465,0.348435


In [39]:
gameoddsxwalk[gameoddsxwalk.isnull().sum(axis=1).astype(bool)]

,hometeam,awayteam,game,season,tie,homewin,draw,awaywin
228,Gent,Wolfsburg,1,2016,gent-wolfsburg,NaN,NaN,NaN
235,Chelsea,Paris SG,2,2016,psg-chelsea,NaN,NaN,NaN


In [32]:
gameoddsxwalk.to_csv('game-odds-xwalk.csv', index=False)